In [1]:
!pip install tf-nightly-2.0-preview


    100% |████████████████████████████████| 75.9MB 448kB/s 
    100% |████████████████████████████████| 3.2MB 7.5MB/s 
    100% |████████████████████████████████| 348kB 8.8MB/s 
    100% |████████████████████████████████| 61kB 22.3MB/s 


In [72]:
import tensorflow as tf
print("You have version", tf.__version__)
assert tf.__version__ >= "2.0" # TensorFlow ≥ 2.0 required

You have version 2.0.0-dev20190207


In [0]:
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten

In [74]:
#load dataset
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255
x_test = x_test / 255

32768/29515=================================] - 0s 0us/step
26427392/26421880==============================] - 0s 0us/step
8192/5148===============================================] - 0s 0us/step
4423680/4422102==============================] - 0s 0us/step


In [0]:
#batch size & shuffle data
BATCH_SIZE = 128
BUFFER_SIZE = len(x_train)

mnist_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
mnist_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

In [0]:
#define model: Linear Model
class MyModel1(Model):
  def __init__(self):
    super(MyModel1, self).__init__()
    self.flatten = Flatten()
    #self.d1 = Dense(128, activation='relu')
    self.d1 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    #x = self.d1(x)
    return self.d1(x)
  
model1 = MyModel1()

In [0]:
#Choose optimizer
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss_metric = tf.keras.metrics.Mean(name='train_loss')
test_loss_metric = tf.keras.metrics.Mean(name='test_loss')

train_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
#Train the model using GradientTape
def train_step(images, labels):
  with tf.GradientTape() as tape:    
    # Forward pass
    predictions = model1(images)
    train_loss = loss_function(y_true=labels, y_pred=predictions)
  
  # Backward pass
  gradients = tape.gradient(train_loss, model1.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model1.trainable_variables))

  # Record results
  train_loss_metric(train_loss)
  train_accuracy_metric(labels, predictions)

In [0]:
def test_step(images, labels):
  predictions = model1(images)
  test_loss = loss_function(y_true=labels, y_pred=predictions)
  
  # Record results
  test_loss_metric(test_loss)
  test_accuracy_metric(labels, predictions)

In [81]:
EPOCHS = 10

for epoch in range(EPOCHS):
  for images, labels in mnist_train:
    train_step(images, labels)
  
  for test_images, test_labels in mnist_test:
    test_step(test_images, test_labels)
  
  template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}, Test loss: {:.4f}, Test accuracy: {:.2f}'
  print (template.format(epoch +1, 
                         train_loss_metric.result(), 
                         train_accuracy_metric.result() * 100, 
                         test_loss_metric.result(), 
                         test_accuracy_metric.result() * 100))

Epoch 1, Loss: 0.7379, Accuracy: 75.67, Test loss: 0.5704, Test accuracy: 80.72
Epoch 2, Loss: 0.6271, Accuracy: 79.25, Test loss: 0.5412, Test accuracy: 81.53
Epoch 3, Loss: 0.5761, Accuracy: 80.85, Test loss: 0.5233, Test accuracy: 82.03
Epoch 4, Loss: 0.5451, Accuracy: 81.81, Test loss: 0.5109, Test accuracy: 82.37
Epoch 5, Loss: 0.5237, Accuracy: 82.47, Test loss: 0.5017, Test accuracy: 82.65
Epoch 6, Loss: 0.5077, Accuracy: 82.96, Test loss: 0.4946, Test accuracy: 82.86
Epoch 7, Loss: 0.4952, Accuracy: 83.33, Test loss: 0.4889, Test accuracy: 83.02
Epoch 8, Loss: 0.4851, Accuracy: 83.64, Test loss: 0.4841, Test accuracy: 83.14
Epoch 9, Loss: 0.4767, Accuracy: 83.89, Test loss: 0.4802, Test accuracy: 83.24
Epoch 10, Loss: 0.4696, Accuracy: 84.11, Test loss: 0.4768, Test accuracy: 83.34


In [0]:
#define model: DNN Model
class MyModel2(Model):
  def __init__(self):
    super(MyModel2, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(64, activation='relu')
    self.d3 = Dense(64, activation='relu')
    self.d4 = Dense(32, activation='relu')
    self.d5 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    x = self.d3(x)
    x = self.d4(x)   
    return self.d5(x)
  
model2 = MyModel2()

In [0]:
#Choose optimizer
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam()

#metric
train_loss_metric = tf.keras.metrics.Mean(name='train_loss')
test_loss_metric = tf.keras.metrics.Mean(name='test_loss')

train_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
#Train the model using GradientTape
def train_step2(images, labels):
  with tf.GradientTape() as tape:    
    # Forward pass
    predictions = model2(images)
    train_loss = loss_function(y_true=labels, y_pred=predictions)
  
  # Backward pass
  gradients = tape.gradient(train_loss, model2.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model2.trainable_variables))

  # Record results
  train_loss_metric(train_loss)
  train_accuracy_metric(labels, predictions)

In [0]:
def test_step2(images, labels):
  predictions = model2(images)
  test_loss = loss_function(y_true=labels, y_pred=predictions)
  
  # Record results
  test_loss_metric(test_loss)
  test_accuracy_metric(labels, predictions)

In [86]:
EPOCHS = 20

for epoch in range(EPOCHS):
  for images, labels in mnist_train:
    train_step2(images, labels)
  
  for test_images, test_labels in mnist_test:
    test_step2(test_images, test_labels)
  
  template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}, Test loss: {:.4f}, Test accuracy: {:.2f}'
  print (template.format(epoch +1, 
                         train_loss_metric.result(), 
                         train_accuracy_metric.result() * 100, 
                         test_loss_metric.result(), 
                         test_accuracy_metric.result() * 100))

Epoch 1, Loss: 0.5836, Accuracy: 79.76, Test loss: 0.4595, Test accuracy: 83.71
Epoch 2, Loss: 0.4927, Accuracy: 82.62, Test loss: 0.4366, Test accuracy: 84.54
Epoch 3, Loss: 0.4468, Accuracy: 84.09, Test loss: 0.4232, Test accuracy: 84.99
Epoch 4, Loss: 0.4166, Accuracy: 85.06, Test loss: 0.4146, Test accuracy: 85.34
Epoch 5, Loss: 0.3948, Accuracy: 85.78, Test loss: 0.4070, Test accuracy: 85.56
Epoch 6, Loss: 0.3774, Accuracy: 86.35, Test loss: 0.4024, Test accuracy: 85.67
Epoch 7, Loss: 0.3632, Accuracy: 86.83, Test loss: 0.3967, Test accuracy: 85.90
Epoch 8, Loss: 0.3510, Accuracy: 87.24, Test loss: 0.3923, Test accuracy: 86.08
Epoch 9, Loss: 0.3405, Accuracy: 87.60, Test loss: 0.3885, Test accuracy: 86.24
Epoch 10, Loss: 0.3309, Accuracy: 87.93, Test loss: 0.3848, Test accuracy: 86.42
Epoch 11, Loss: 0.3227, Accuracy: 88.21, Test loss: 0.3823, Test accuracy: 86.56
Epoch 12, Loss: 0.3153, Accuracy: 88.46, Test loss: 0.3805, Test accuracy: 86.68
Epoch 13, Loss: 0.3083, Accuracy: 88.